# Import libraries

In [26]:
import os
import cv2
import glob
import tqdm
import json
import pprint
import numpy as np


## 1. Create annotation dictionary

In [3]:
classes = ['car', 'motorcycle', 'truck', 'bus', 'bicycle']
annotations = {x:{} for x in classes}
print(annotations)

{'car': {}, 'motorcycle': {}, 'truck': {}, 'bus': {}, 'bicycle': {}}


## 2. Loop through the annotation files

In [8]:
# Check how many annotation and image files there is
annot_files = glob.glob('./labels/*.txt')
img_files = glob.glob('./images/*.jpg') + glob.glob('./images/*.jpeg') + glob.glob('./images/*.png')

print(f'[INFO] {len(annot_files)} Annotation files')
print(f'[INFO] {len(img_files)} Image files')

[INFO] 1142 Annotation files
[INFO] 1196 Image files


In [13]:
# Join the two set of annotations and images together
excluded = 0
for annot in annot_files:
    # Get the filename
    filename = annot.split('/')[-1].split('.')[0]
    
    # Check if corresponding image file exists
    if(f'./images/{filename}.jpg' not in img_files and 
       f'./images/{filename}.png' not in img_files and
       f'./images/{filename}.jpeg' not in img_files):
        excluded += 1
        
        print('[INFO] Removing ', annot)
        os.remove(annot)
        
for img in img_files:
    # Get the filename
    filename = img.split('/')[-1].split('.')[0]
    
    # Check if the corresponding annotation file exists
    if(f'./labels/{filename}.txt' not in annot_files):
        excluded += 1
        print('[INFO] Removing ', img)
        os.remove(img)
        
print(f'[INFO]  -->  {excluded} files excluded')

[INFO] Removing  ./images/35.jpg
[INFO] Removing  ./images/520.jpg
[INFO] Removing  ./images/521.jpg
[INFO] Removing  ./images/33.jpg
[INFO] Removing  ./images/32.jpg
[INFO] Removing  ./images/400.jpg
[INFO] Removing  ./images/22.jpg
[INFO] Removing  ./images/577.jpg
[INFO] Removing  ./images/436.jpg
[INFO] Removing  ./images/345.jpg
[INFO] Removing  ./images/326.jpg
[INFO] Removing  ./images/329.jpg
[INFO] Removing  ./images/339.jpg
[INFO] Removing  ./images/90.jpg
[INFO] Removing  ./images/1001.jpeg
[INFO] Removing  ./images/1037.jpeg
[INFO] Removing  ./images/1013.jpeg
[INFO] Removing  ./images/1033.jpeg
[INFO] Removing  ./images/1014.jpeg
[INFO] Removing  ./images/1023.jpeg
[INFO] Removing  ./images/1021.jpeg
[INFO] Removing  ./images/1034.jpeg
[INFO] Removing  ./images/1035.jpeg
[INFO] Removing  ./images/1012.jpeg
[INFO] Removing  ./images/1011.jpeg
[INFO] Removing  ./images/1015.jpeg
[INFO] Removing  ./images/1027.jpeg
[INFO] Removing  ./images/1036.jpeg
[INFO] Removing  ./images

## 3. Create annotation in JSON

In [22]:
# Create a text to image dictionary
img_annot = {}
for img in img_files:
    # Get the filename
    filename = img.split('/')[-1].split('.')[0]
    
    # Get the corresponding text annotation
    annot = f'./labels/{filename}.txt'
    if(annot not in annot_files):
        continue
    
    img_annot[annot] = img

In [30]:
# Create annotation dictionary
classes = ['car', 'motorcycle', 'truck', 'bus', 'bicycle']
annotations = {x:{} for x in classes}

with tqdm.tqdm(total=len(img_annot.keys())) as pbar:
    for annot in img_annot.keys():
        with open(annot, 'r') as annot_f:
            labels = annot_f.read().strip().split('\n')
            img_file = img_annot[annot]

            # Read the image and get dimensions
            img = cv2.imread(img_file)
            H, W = img.shape[:2]

            for label in labels:
                tokens = label.split(' ')
                class_name = classes[int(tokens[0])]
                x, y, w, h = float(tokens[1]), float(tokens[2]), float(tokens[3]), float(tokens[4])
                x1, y1, x2, y2 = int(x*W), int(y*H), int((x+w)*W), int((y+h)*H)

                if(img_file not in annotations[class_name]):
                    annotations[class_name][img_file] = []

                annotations[class_name][img_file].append([x1, y1, x2, y2])
        pbar.update(1)
            
pprint.pprint(annotations)

100%|██████████| 1142/1142 [00:07<00:00, 157.61it/s]


{'bicycle': {'./images/341.jpg': [[1129, 311, 1208, 470]],
             './images/image105.jpg': [[102, 230, 184, 552]],
             './images/image110.jpg': [[195, 227, 484, 564]],
             './images/image121.jpg': [[211, 211, 476, 617]],
             './images/image131.jpg': [[111, 246, 194, 558],
                                       [255, 257, 338, 561]],
             './images/image134.jpg': [[244, 210, 440, 592]],
             './images/image136.jpg': [[196, 213, 446, 616]],
             './images/image138.jpg': [[176, 205, 496, 614]],
             './images/image155.jpg': [[268, 209, 306, 299]],
             './images/image17.jpg': [[212, 260, 408, 528]],
             './images/image170.jpg': [[165, 218, 430, 611]],
             './images/image176.jpg': [[207, 190, 454, 495]],
             './images/image187.jpg': [[261, 171, 394, 512]],
             './images/image189.jpg': [[241, 242, 566, 536]],
             './images/image198.jpg': [[222, 196, 511, 564]],
             

           './images/img696.jpg': [[93, 273, 178, 317],
                                   [146, 272, 210, 340],
                                   [133, 216, 165, 236],
                                   [258, 211, 293, 228]],
           './images/img707.jpg': [[201, 208, 603, 576]],
           './images/img71.jpg': [[33, 83, 99, 134],
                                  [65, 122, 196, 249],
                                  [151, 130, 446, 295],
                                  [243, 167, 588, 475],
                                  [356, 263, 475, 509]],
           './images/img732.jpg': [[250, 222, 580, 568]],
           './images/img747.jpg': [[264, 261, 499, 449]],
           './images/img779.jpg': [[237, 88, 385, 182]],
           './images/img788.jpg': [[236, 239, 586, 554]],
           './images/img799.jpg': [[54, 300, 160, 467],
                                   [66, 136, 198, 247],
                                   [144, 123, 431, 237]],
           './images/img803.jpg': [[

In [33]:
# Save the annotations
with open('../predictions/gt_vehicle.json', 'w') as f:
    json.dump(annotations, f, indent=4)